In [ ]:
# This notebook consists of actions for preparation for Horizon tests
# It creates a fake cube with horizon
import os
import sys
import shutil
from datetime import date
import warnings
import numpy as np

warnings.filterwarnings("ignore")

sys.path.append('../../../seismiqb')

from seismiqb.src.synthetic import generate_synthetic
from seismiqb.src.geometry.export import make_segy_from_array
from seismiqb import Horizon, Field

In [ ]:
""" You can manage notebook execution kwargs which relates to cube and horizon for the test:

SYNTHETIC_MODE : bool
    Whether to create a synthetic data (cube and horizon) or use existed, provided by paths.
CUBE_PATH : str or None
    Path to an existed seismic cube.
    Notice that it is only used with SYNTHETIC_MODE = False.
HORIZON_PATH : str or None
    Path to an existed seismic horizon.
    Notice that it is only used with SYNTHETIC_MODE = False.
CUBE_SHAPE : sequence of three integers
    Shape of a synthetic cube.
GRID_SHAPE: sequence of two integers
    Sets the shape of grid of support points for surfaces' interpolation
    (surfaces represent horizons).
SEED: int or None
    Seed used for creation of random generator (check out `np.random.default_rng`).
"""

# Workspace constants
DATESTAMP = date.today().strftime("%Y-%m-%d")
TESTS_ROOT_DIR = './'

# Synthetic creation parameters
SYNTHETIC_MODE = True
CUBE_PATH = None
HORIZON_PATH = None
CUBE_SHAPE = (500, 500, 200)
GRID_SHAPE = (10, 10)
SEED = 42

# Visualization parameters
SCALE = 1
SHOW_FIGURES = False

# Preparation

Create directories for files and create a fake cube with horizons and save cube and one horizon.

**Storage structure:**
___



**horizon_test_files** (tests root directory)

&emsp;├── **test_cube.sgy**

&emsp;├── **test_cube.meta**
    
&emsp;└── **test_horizon**


In [ ]:
# (Re)create the test directory
OUTPUT_DIR = os.path.join(TESTS_ROOT_DIR, "horizon_test_files")

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

In [ ]:
%%time
# Synthetic data creation
if SYNTHETIC_MODE:
    CUBE_PATH = os.path.join(OUTPUT_DIR, f"test_cube_{DATESTAMP}.sgy")
    HORIZON_PATH = os.path.join(OUTPUT_DIR, f"test_horizon_{DATESTAMP}")

    # Create a cube and save it
    synt3d, hors, _ = generate_synthetic(shape=CUBE_SHAPE, grid_shape=GRID_SHAPE,
                                         num_reflections=60,
                                         geobodies_format=('heights', 'mask'), seed=SEED)

    # Create zero traces in the cube (for testing filetring)
    FILL_VALUE = -999999
    points = (
        slice(0, CUBE_SHAPE[0]//10, None),
        slice(CUBE_SHAPE[0]//10, CUBE_SHAPE[0]//5, None),
        slice(None)
    )
    synt3d[points] = FILL_VALUE

    make_segy_from_array(synt3d, CUBE_PATH, zip_segy=False)

    # Check cube
    field = Field(CUBE_PATH)

    assert np.allclose(field.geometry[:, :, :], synt3d)

    # Choose one horizon and save it
    horizons = [Horizon(hor, field, 'synthetic_' + str(i)) for i, hor in enumerate(hors)]
    horizon = horizons[0]

    horizon.dump(HORIZON_PATH)

    # Check horizon
    opened_horizon = Horizon(HORIZON_PATH, field=field)

    assert np.array_equal(horizon.matrix, opened_horizon.matrix)
    assert np.array_equal(horizon.points, opened_horizon.points)

    opened_horizon.filter()
    opened_horizon.show(show=SHOW_FIGURES, scale=SCALE)